Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2017.2

Professor: André Lage Freitas


# Trabalho Prático AB2


## Especificação

_Importante_. Leia atentamente essa seção pois só serão aceitos trabalhos que obedecerem as a especificações a seguir. 

**Implementar o detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Os **_workers_ devem enviar ao _master_ a cada segundo o estado dos workers** que eles monitoram, indicando quais _workers_ estão suspeitos ou não suspeitos.

### Testando o trabalho

Exemplo de como testar o detector de falhas em sistemas UNIX:

```bash
kill -9 $(ps aux | grep -i julia | grep worker | awk '{print $2}' | head -1 )
```


### Algumas funções úteis

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na implementação:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize a [documentação oficial da linguagem Julia (0.6.2)](https://docs.julialang.org/en/stable/) para demais dúvidas.

## Código-fonte

In [ ]:
#=
No código a seguir, todos os processos vigiam todos os processos. Cada processo manda "estou vivo" para todos os outros
processos, e então dorme por 1 segundo. Cada processo que recebe um "estou vivo" registra o tempo atual para aquele
processo.

Após dormir, cada processo checa possíveis suspeitos. Para isso, ele verifica se o tempo da última vez que um processo
disse "estou vivo" é maior ou igual que o tempo de soneca + tempo máximo de espera. Por exemplo, se fazem 5 segundos
que o processo disse estar vivo, mas o tempo de soneca + tempo máximo de espera é igual a 3, então esse processo é
suspeito.
=#

# adiciona processos
addprocs(4)

# define as variáveis a serem manipuladas por cada processo
# lista de ids de suspeitos
@everywhere suspects = Int[]
# ids dos processos
@everywhere workers_ids = Int[]
# id do processo
@everywhere my_id = Int
# dicionário mapeia id dos processos e tempo que disse estar vivo pela última vez
@everywhere time_alive = Dict()
# tempo máximo de espera
@everywhere max_time = 2
# tempo que um processo dorme
@everywhere sleep_time = 1

# informa aos outros processos que está vivo
@everywhere function send_alive()
    global workers_ids
    global my_id
    
    for id in workers_ids
        if id != my_id
            try
                fetch(remotecall(alive, id, my_id))
            end
        end
    end
end

# função ativada quando um processo é informado que outro está vivo
# id: id do outro processo que diz estar vivo
@everywhere function alive(id)
    global suspects
    global time_alive
    
    # registra o tempo atual para o processo
    time_alive[id] = time()
    # remove dos suspeitos, se tiver lá
    deleteat!(suspects, findin(suspects, id))
end

# inicialização
# all_ids: lista com ids de todos os processos
# worker_id: id do processo sendo inicializado
@everywhere function init(all_ids, worker_id)
    global my_id = worker_id
    global workers_ids = all_ids
    global time_alive
    
    for id in all_ids
        time_alive[id] = time()
    end
end

# checa à procura de suspeitos
@everywhere function check_suspects()
    global my_id
    global workers_ids
    global suspects
    global sleep_time
    global max_time
    global time_alive
    
    for id in workers_ids
        if id == my_id
            continue
        end
        
        # se já faz muito tempo que um processo informou que está vivo, coloca como suspeito
        if ((time() - time_alive[id]) >= (max_time + sleep_time))
            if length(findin(suspects, id)) == 0
                push!(suspects, id)
            end
        # caso contrário, retirar dos suspeitos, se estiver lá
        else
            deleteat!(suspects, findin(suspects, id))
        end
    end
end

# o loop principal do processo
@everywhere function main_loop()
    global suspects
    global my_id
    global sleep_time
    
    while true
        # envia "estou vivo"
        send_alive()
        
        # dorme
        sleep(sleep_time)
        
        # checa suspeitos
        check_suspects()
        
        # informa ao master os suspeitos
        try
            fetch(remotecall(print_suspects, 1, my_id, suspects))
        end
    end
end

# função usada somente na master
# informa os suspeitos de um worker
# work_id: id do worker
# suspects_: suspeitos do worker
@everywhere function print_suspects(work_id, suspects_)
    println("suspects from worker $work_id : $suspects_")
end

# inicializa todos os workers
for id in workers()
    try
        fetch(remotecall(init, id, workers(), id))
    catch
        println("error to initialize worker number $id")
    end
end

# inicia o loop principal de cada worker
for id in workers()
    try
        @async fetch(remotecall(main_loop, id))
    end
end


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org). Utilize explicações sobre o código em [Markdown](https://en.wikipedia.org/wiki/Markdown).

O programa deve ser implementado na linguagem de programação [**Julia** versão 0.6.2](https://docs.julialang.org/en/stable/). Seu trabalho será testado na [JuliaBox](https://juliabox.com), portanto teste seu arquivo `.ipynb` IJulia na JuliaBox antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.